What This Script Does

Loads TinyLLaMA (1.1B) in 4-bit → fits your 6GB GPU.

Creates a tiny JSONL dataset (3 samples).

Formats data in Alpaca-style (Instruction, Input, Response).

Applies LoRA tuning (very lightweight).

Trains for 1 epoch (finishes in a few minutes).

Evaluates loss on same tiny dataset.

Runs inference to test tuned behavior.

Prints hyperparameter tips + impact.

In [1]:
# ================================
# 1) Imports & Setup
# ================================
import os
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer,
    TrainingArguments, Trainer,
    DataCollatorForLanguageModeling
)
from peft import (
    LoraConfig, get_peft_model,
    prepare_model_for_kbit_training, PeftModel
)

# Model & paths
MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
OUTPUT_DIR = "tiny_inst_lora"
JSONL = "inst_tiny.jsonl"

C:\Users\abhishek jaiswal\inkri\qualcomm_tasks\llama_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ================================
# 2) Create Tiny Dataset (reduced size)
# ================================
if not os.path.exists(JSONL):
    with open(JSONL, "w", encoding="utf-8") as f:
        f.write('{"instruction":"Summarize in one sentence.","input":"The sun warms the Earth and helps plants grow.","output":"The sun provides warmth and energy for plant growth."}\n')
        f.write('{"instruction":"Translate to French.","input":"Good morning, how are you?","output":"Bonjour, comment ça va ?"}\n')
        f.write('{"instruction":"List two fruits.","input":"","output":"Apple, Banana"}\n')

raw_dataset = load_dataset("json", data_files=JSONL, split="train")

Generating train split: 3 examples [00:00, 292.81 examples/s]


In [3]:
# ================================
# 3) Load Model & Tokenizer (4-bit)
# ================================
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    load_in_4bit=True
)
model = prepare_model_for_kbit_training(model)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
The 8-bit optimizer is not available on your device, only available on CUDA for now.


In [4]:
# ================================
# 4) Apply LoRA Config
# ================================
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # lightweight
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

In [5]:
# ================================
# 5) Format Data (prompt template)
# ================================
def format_example(example):
    prompt = f"### Instruction:\n{example['instruction']}\n"
    if example["input"]:
        prompt += f"### Input:\n{example['input']}\n"
    prompt += "### Response:\n"
    full_text = prompt + example["output"]
    return {"text": full_text}

formatted = raw_dataset.map(format_example)

def tokenize_function(example, max_length=256):
    return tokenizer(
        example["text"],
        truncation=True,
        max_length=max_length,
        padding="max_length"
    )

tokenized = formatted.map(tokenize_function, remove_columns=formatted.column_names)

Map: 100%|███████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 215.38 examples/s]


In [6]:
# ================================
# 6) Training
# ================================
collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=torch.cuda.is_available(),
    logging_steps=1,
    save_strategy="no",
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized,
    data_collator=collator
)

trainer.train()

# Save adapter
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("✅ Training finished. Adapter saved to", OUTPUT_DIR)

C:\Users\abhishek jaiswal\inkri\qualcomm_tasks\llama_env\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
C:\Users\abhishek jaiswal\inkri\qualcomm_tasks\llama_env\Lib\site-packages\torch\_dynamo\eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,2.520800


✅ Training finished. Adapter saved to tiny_inst_lora


In [7]:
# ================================
# 7) Evaluation (loss)
# ================================
eval_args = TrainingArguments(
    output_dir="eval_tmp",
    per_device_eval_batch_size=1,
)
trainer = Trainer(
    model=model,
    args=eval_args,
    eval_dataset=tokenized,
    data_collator=collator
)
metrics = trainer.evaluate()
print("📊 Eval metrics:", metrics)

📊 Eval metrics: {'eval_loss': 2.645254135131836, 'eval_model_preparation_time': 0.0062, 'eval_runtime': 17.2008, 'eval_samples_per_second': 0.174, 'eval_steps_per_second': 0.174}


In [8]:
# ================================
# 8) Inference with Fine-tuned Model
# ================================
def make_prompt(instruction, inp=""):
    txt = f"### Instruction:\n{instruction}\n"
    if inp:
        txt += f"### Input:\n{inp}\n"
    txt += "### Response:\n"
    return txt

# Reload model + adapter
base = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    load_in_4bit=True
)
model = PeftModel.from_pretrained(base, OUTPUT_DIR)
model.eval()

prompt = make_prompt("Translate to Spanish.", "I love learning with small models.")
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
out = model.generate(**inputs, max_new_tokens=60, do_sample=True, temperature=0.7)
print("💡 Model output:\n", tokenizer.decode(out[0], skip_special_tokens=True))

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


💡 Model output:
 ### Instruction:
Translate to Spanish.
### Input:
I love learning with small models.
### Response:
Me encanta aprender con modelos pequeños.
### Notes:
- The original text is in English. The translated text is in Spanish.


In [10]:
# ================================
# 8.1) Inference with Fine-tuned Model
# ================================
from transformers import AutoModelForCausalLM
from peft import PeftModel

def make_prompt(instruction, inp=""):
    txt = f"### Instruction:\n{instruction}\n"
    if inp:
        txt += f"### Input:\n{inp}\n"
    txt += "### Response:\n"
    return txt

# Reload model + adapter
base = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    load_in_4bit=True
)
model = PeftModel.from_pretrained(base, OUTPUT_DIR)
model.eval()

# List of test instructions
test_cases = [
    {
        "instruction": "Translate to Spanish.",
        "input": "I love learning with small models."
    },
    {
        "instruction": "Summarize in one sentence.",
        "input": "The sun warms the Earth and helps plants grow."
    },
    {
        "instruction": "Translate to French.",
        "input": "Good morning, how are you?"
    },
    {
        "instruction": "List two fruits.",
        "input": ""
    }
]

# Run inference for each test case
for case in test_cases:
    prompt = make_prompt(case["instruction"], case["input"])
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(
        **inputs,
        max_new_tokens=60,
        do_sample=True,
        temperature=0.7
    )
    print("=======================================")
    print(f"Instruction: {case['instruction']}")
    if case["input"]:
        print(f"Input: {case['input']}")
    print("💡 Model output:\n", tokenizer.decode(out[0], skip_special_tokens=True))
    print("=======================================\n")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Instruction: Translate to Spanish.
Input: I love learning with small models.
💡 Model output:
 ### Instruction:
Translate to Spanish.
### Input:
I love learning with small models.
### Response:
Enamorándome aprendiendo con modelos pequeños.

Instruction: Summarize in one sentence.
Input: The sun warms the Earth and helps plants grow.
💡 Model output:
 ### Instruction:
Summarize in one sentence.
### Input:
The sun warms the Earth and helps plants grow.
### Response:
The sun's energy is used to power the Earth's processes, which helps plants grow.

Instruction: Translate to French.
Input: Good morning, how are you?
💡 Model output:
 ### Instruction:
Translate to French.
### Input:
Good morning, how are you?
### Response:
Je m'appelle Sophie et je suis désormais en vacances, je vais prendre des vacances trois semaines conséquent.

### Output:
Bonjour, comment ça est?
### French:
Je m'appelle Soph

Instruction: List two fruits.
💡 Model output:
 ### Instruction:
List two fruits.
### Response:


In [9]:
# ================================
# 9) Hyperparameter Tips
# ================================
print("""
👉 Hyperparameter notes:
- EPOCHS=1 for demo; increase to 3–5 for better results.
- LoRA r=4 (small); increase to 8–16 for stronger adaptation.
- MAX_LENGTH=256 to fit RTX 4050; can try 512 with gradient checkpointing.
- Dataset only 3 samples for speed; add more for real tuning.
""")

# ================================
# 10) Post-Tuning Impact
# ================================
print("""
✅ Post-Tuning Impact:
- Model now respects instruction templates (Instruction, Input, Response).
- Even with 3 samples, behavior shifts toward following instructions.
- With more data, this method can adapt LLaMA models to domains/tasks.
""")


👉 Hyperparameter notes:
- EPOCHS=1 for demo; increase to 3–5 for better results.
- LoRA r=4 (small); increase to 8–16 for stronger adaptation.
- MAX_LENGTH=256 to fit RTX 4050; can try 512 with gradient checkpointing.
- Dataset only 3 samples for speed; add more for real tuning.


✅ Post-Tuning Impact:
- Model now respects instruction templates (Instruction, Input, Response).
- Even with 3 samples, behavior shifts toward following instructions.
- With more data, this method can adapt LLaMA models to domains/tasks.



In [ ]:
#Thank you